In [2]:
#使用TF重复卷积过程
import tensorflow as tf
import numpy as np

x_input = np.array([
    [1,1,1,0,0],
    [0,1,1,1,0],
    [0,0,1,1,1],
    [0,0,1,1,0],
    [0,1,1,0,0]
],dtype=np.float32)

x_kernel_1 = np.array([
    [1,0,1],
    [0,1,0],
    [1,0,1]
],dtype = np.float32)

tf_x_input = tf.constant(np.reshape(x_input,newshape=[1,5,5,1]))
tf_x_kernel_1 = tf.constant(np.reshape(x_kernel_1,newshape=[3,3,1,1]))

y1 = tf.nn.conv2d(tf_x_input,tf_x_kernel_1,strides=[1,1,1,1],padding="VALID")
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    [y1_cov] = sess.run([y1])
    
y1_cov[0,:,:,0]

array([[4., 3., 4.],
       [2., 4., 3.],
       [2., 3., 4.]], dtype=float32)

In [3]:
#使用第二种卷积核的输出y2_cov
x_kernel_2 = np.array([
    [0,1,0],
    [1,0,1],
    [0,1,0]
],dtype=np.float32)

tf_x_kernel_2 = tf.constant(np.reshape(x_kernel_2,newshape=[3,3,1,1]))
y2 = tf.nn.conv2d(tf_x_input,tf_x_kernel_2,strides=[1,1,1,1],padding="VALID")
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    [y1_cov,y2_cov] = sess.run([y1,y2])
print(u"第一种扫描结果：")
print(y1_cov[0,:,:,0])
print(u"第二种扫描结果：")
print(y2_cov[0,:,:,0])

第一种扫描结果：
[[4. 3. 4.]
 [2. 4. 3.]
 [2. 3. 4.]]
第二种扫描结果：
[[2. 4. 2.]
 [2. 3. 4.]
 [2. 3. 2.]]


In [4]:
#输入层加一个图层，里面内容同之前，这样输入维度增加到2
x_input2 = np.zeros([1,5,5,2])
x_input2[0,:,:,0] = x_input
x_input2[0,:,:,1] = x_input

#输入维度增加到2，卷积核输入维度同样做增加，但这里输出仍然是一个维度
x_kernel_3 = np.zeros([3,3,2,1])
x_kernel_3[:,:,0,0] = x_kernel_1
x_kernel_3[:,:,1,0] = x_kernel_2

tf_x_input2 = tf.constant(x_input2.astype(np.float32))
tf_x_kernel_3 = tf.constant(x_kernel_3.astype(np.float32))

y3 = tf.nn.conv2d(tf_x_input2,tf_x_kernel_3,strides=[1,1,1,1],padding="VALID")
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    [y3_cov] = sess.run([y3])
    
#发现这里卷积层输出内容是单独两个卷积核扫描结果的直接相加
print(u"第一，第二种卷积核扫描结果简单相加")
print((y1_cov+y2_cov)[0,:,:,0])

print(u"第一，第二种卷积核组合后扫描两层相同输入图层结果")
print(y3_cov[0,:,:,0])

第一，第二种卷积核扫描结果简单相加
[[6. 7. 6.]
 [4. 7. 7.]
 [4. 6. 6.]]
第一，第二种卷积核组合后扫描两层相同输入图层结果
[[6. 7. 6.]
 [4. 7. 7.]
 [4. 6. 6.]]


In [5]:
#现在将卷积层输出增加到2个，即卷积核的输出维度为2
#输出层使用相同的卷积
x_kernel_4=np.zeros([3,3,2,2])
x_kernel_4[:,:,0,0]=x_kernel_1
x_kernel_4[:,:,1,0]=x_kernel_1
x_kernel_4[:,:,0,1]=x_kernel_2
x_kernel_4[:,:,1,1]=x_kernel_2

#输入层使用相同的卷积
x_kernel_5=np.zeros([3,3,2,2])
x_kernel_5[:,:,0,0]=x_kernel_1
x_kernel_5[:,:,0,1]=x_kernel_1
x_kernel_5[:,:,1,0]=x_kernel_2
x_kernel_5[:,:,1,1]=x_kernel_2

tf_x_kernel_4 = tf.constant(x_kernel_4.astype(np.float32))
tf_x_kernel_5 = tf.constant(x_kernel_5.astype(np.float32))
y4 = tf.nn.conv2d(tf_x_input2,tf_x_kernel_4,strides=[1,1,1,1],padding="VALID")
y5 = tf.nn.conv2d(tf_x_input2,tf_x_kernel_5,strides=[1,1,1,1],padding="VALID")
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    [y4_cov,y5_cov] =sess.run([y4,y5])
    
print(u"输出层相同卷积核结果")
print(y4_cov[0,:,:,0])
print(y4_cov[0,:,:,1])

print(u"输入层用相同卷积核的结果")
print(y5_cov[0,:,:,0])
print(y5_cov[0,:,:,1])

输出层相同卷积核结果
[[8. 6. 8.]
 [4. 8. 6.]
 [4. 6. 8.]]
[[4. 8. 4.]
 [4. 6. 8.]
 [4. 6. 4.]]
输入层用相同卷积核的结果
[[6. 7. 6.]
 [4. 7. 7.]
 [4. 6. 6.]]
[[6. 7. 6.]
 [4. 7. 7.]
 [4. 6. 6.]]


In [8]:
#UpSampling2D(2,2)的用法核MaxPlloing2D基本相反
#UpSampling2D(2,2)是将输入图片长宽各拉伸1倍
#现在使用一个3*3的全是1的卷积核对输入层执行反卷积操作
x_kernel_3=np.array([
    [1,1,1],
    [1,1,1],
    [1,1,1]
],dtype=np.float32)
tf_x_kernel_3=tf.constant(np.reshape(x_kernel_3,newshape=[3,3,1,1]))

y1_trans = tf.nn.conv2d_transpose(y1,tf_x_kernel_3,output_shape=[1,5,5,1],strides=[1,2,2,1],padding="SAME")#反卷积

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    [y1_cov_tran]=sess.run([y1_trans])
    
print(u"反卷积输入")
print(y1_cov[0,:,:,0])
print(u"反卷积输出")
print(y1_cov_tran[0,:,:,0])

反卷积输入
[[4. 3. 4.]
 [2. 4. 3.]
 [2. 3. 4.]]
反卷积输出
[[ 4.  7.  3.  7.  4.]
 [ 6. 13.  7. 14.  7.]
 [ 2.  6.  4.  7.  3.]
 [ 4. 11.  7. 14.  7.]
 [ 2.  5.  3.  7.  4.]]


In [18]:
x_input_tran_zero=np.array([
    [4,0,3,0,4],
    [0,0,0,0,0],
    [2,0,4,0,3],
    [0,0,0,0,0],
    [2,0,3,0,4]
],dtype=np.float32)
tf_input_tran_zero=tf.constant(np.reshape(x_input_tran_zero,newshape=[1,5,5,1]))
y1_trans_zero = tf.nn.conv2d(tf_input_tran_zero,tf_x_kernel_3,strides=[1,2,2,1],padding="SAME")#卷积
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    [y1_cov_tran2]=sess.run([y1_trans_zero])
print(u"反卷积输出")
print(y1_cov_tran2[0,:,:,0])

反卷积输出
[[4. 3. 4.]
 [2. 4. 3.]
 [2. 3. 4.]]


In [35]:
y1_trans=tf.nn.conv2d_transpose(y1,tf_x_kernel_3,output_shape=[1,9,9,1],strides=[1,4,4,1],padding="SAME")
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    [y1_cov_tran]=sess.run([y1_trans])
print(u"反卷积输入")
print(y1_cov[0,:,:,0])
print(u"反卷积输出")
print(y1_cov_tran[0,:,:,0])

反卷积输入
[[4. 3. 4.]
 [2. 4. 3.]
 [2. 3. 4.]]
反卷积输出
[[4. 4. 0. 3. 3. 3. 0. 4. 4.]
 [4. 4. 0. 3. 3. 3. 0. 4. 4.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [2. 2. 0. 4. 4. 4. 0. 3. 3.]
 [2. 2. 0. 4. 4. 4. 0. 3. 3.]
 [2. 2. 0. 4. 4. 4. 0. 3. 3.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [2. 2. 0. 3. 3. 3. 0. 4. 4.]
 [2. 2. 0. 3. 3. 3. 0. 4. 4.]]
